## SD on EKS 部署
以下为快速部署版本,一些过程如果不能在noteb cell中执行,请粘贴相关内容到shell(bash)中执行,很多参数未做变量,请仔细修改

### 一. 基本环境初始化

#### 1. 安装基本命令

%%bash
sudo chmod +x init-env.sh
./init-env.sh

#### 2. clone 代码

In [ ]:
%%bash
git clone --recurse-submodules https://Private_key@github.com/bnusunny/stable-diffusion-on-eks.git
cd ~/SageMaker/SDonEKS-Deploy/stable-diffusion-on-eks

#### 3. 创建模型存储桶,记录下桶的arn 后面会用到

In [ ]:
! aws s3api create-bucket --bucket model-store --region <Your_Region>

### 二. 镜像及Snapshot制备

#### 1. 镜像制作

In [ ]:
!docker build -t inference-api:latest src/sd_webui_api/

In [ ]:
!docker build -t queue-agent:latest src/queue_agent/

In [ ]:
%%bash
aws ecr create-repository --repository-name sd-on-eks/inference-api
aws ecr create-repository --repository-name sd-on-eks/queue-agent

In [ ]:
%%bash
aws ecr get-login-password --region <Your_Region> | docker login --username AWS --password-stdin <Your_Account>.dkr.ecr.<Your_Region>.amazonaws.com
docker tag inference-api:latest <Your_Account>.dkr.ecr.<Your_Region>.amazonaws.com/sd-on-eks/inference-api:latest
docker push <Your_Account>.dkr.ecr.<Your_Region>.amazonaws.com/sd-on-eks/inference-api:latest
docker tag queue-agent:latest <Your_Account>.dkr.ecr.<Your_Region>.amazonaws.com/sd-on-eks/queue-agent:latest
docker push <Your_Account>.dkr.ecr.<Your_Region>.amazonaws.com/sd-on-eks/queue-agent:latest

In [ ]:
#记录下上面输出的image URL

In [ ]:
!aws ecr create-repository --repository-name sd-on-eks/charts/sd-on-eks

In [ ]:
helm package src/eks_cluster/charts/sd-on-eks
helm push sd-on-eks-<version>.tgz oci://<Your_Account>.dkr.ecr.<Your_Region>.amazonaws.com/sd-on-eks/charts/

In [ ]:
#记录下上面输出的oci URL

#### 2. Snapshot 制作

In [ ]:
%%bash
cd /home/ec2-user/SageMaker/SDonEKS-Deploy/stable-diffusion-on-eks/utils/bottlerocket-images-cache
./snapshot.sh <Your_Account>.dkr.ecr.<Your_Region>.amazonaws.com/sd-on-eks/inference-api:latest,<Your_Account>.dkr.ecr.<Your_Region>.amazonaws.com/sd-on-eks/queue-agent:latest

In [ ]:
#记录下上面输出的snapshot id

### 三. CDK 部署

#### 修改 config.yaml文件,参考编辑Sample目录下的config.yaml文件,注意标点和格式

根据实际情况修改,S3桶arn,runtime 模型,自建ECR,Snapshot ID等

In [ ]:
# 保存后执行以下步骤开始部署
%%bash
npm install
cdk bootstrap
cdk deploy --all --no-roll-back

执行完毕后从cloudformation输出记录 APIGateway 地址,形如https://c1544mqj.execute-api.<Your_Region>.amazonaws.com/prod/

执行完毕后从cloudformation输出记录 eks config update命令,如下,拷贝到bash 中执行:

```
aws eks update-kubeconfig --name SdOnEksDataPlaneStack --region <Your_Region> --role-arn arn:aws:iam::<Your_Account>:role/SdOnEksDataPlaneStack-SdOnEksDataPlaneStackAccessR-XXXXX
```

### 四. 模型下载同步

#### 1. 模型下载

找到tools目录下S3uploader文件夹,修改其中的down.csv 修改其中的url,bucket和目标key列后保存
执行

```
python3 s3uploaderv2.py down.csv
```
开始下载

#### 2. 模型同步

下载tools目录下下载models_sync_deploy.yaml并在cloudformation中创建堆栈,桶输入之前的模型桶,efsid 选择cloudformation中创建的EFSid,网络选择CDK创建的任意子网,安全组选择任意可以访问efs的安全组(如果没有可以手动创建一个)

Models_sync_deploy创建完成后,可以到datasync 控制台中找到历史记录是否有执行记录,如果等待5分钟仍然没有,可以找到任务的开始按钮,点击如提示错误,在编辑下给与Cloudwatch 日志组权限,等待片刻再观察.

#### 五.排错及管理

1. 首批pod可能由于模型加载错误导致无法启动可以使用如下命令重启pod

```
kubectl scale deployment sdruntime2-sd-on-eks-sd-webui-inference-api --replicas=0 -n sdruntime2
kubectl scale deployment sdruntime1-sd-on-eks-sd-webui-inference-api --replicas=0 -n sdruntime1
kubectl scale deployment sdruntime1-sd-on-eks-sd-webui-inference-api --replicas=1 -n sdruntime1
kubectl scale deployment sdruntime2-sd-on-eks-sd-webui-inference-api --replicas=1 -n sdruntime2
```

2. 如果Pod处于pending状态,请检查一下命令输出是否有异常

```
kc describe pod sdruntime1-sd-on-eks-sd-webui-inference-api-74f7cb554c-s6jh4 -n sdruntime1
kubectl logs -n karpenter -l app.kubernetes.io/name=karpenter
kubectl logs karpenter-76c95bd775-rdw6q  -n karpenter
```

3. 如果Pod启动出错,请检查一下命令输出是否有异常

```
kubectl logs sdruntime1-sd-on-eks-sd-webui-inference-api-74f7cb554c-t47rq -c sd-webui-queue-agent -n sdruntime1
kubectl exec -it  sdruntime1-sd-on-eks-sd-webui-inference-api-cb57f64fd-gt48p -c sd-webui-queue-agent -n sdruntime1 /bin/bash
kubectl logs sdruntime1-sd-on-eks-sd-webui-inference-api-cb57f64fd-gt48p -c sd-webui-queue-agent -n sdruntime2
```

#### 六.请求测试

1. 使用DemoUI目录下 demo.py进行测试

2. 使用API_Test.ipynb进行测试